In [1]:
import numpy as np
import robustsp as rsp
from scipy.optimize import least_squares as lsq

def arma_est_bip_s(x,p,q):
    
    if p==0 and q == 0:
        inno_scale = rsp.m_scale(x)
        print('at least p or q has to be non-zero')
    if len(x) <= p+q:
        raise ValueError('There are too many parameters to estimate for chosen data size. Reduce model order or use a larger data set.')
    
    # Robust starting point by BIP AR-S approximation
    beta_initial = rsp.robust_starting_point(x,p,q)[0]
    
    F = lambda beta: rsp.arma_s_resid_sc(x, beta, p, q)
    
    F_bip = lambda beta: rsp.bip_s_resid_sc(x, beta, p, q)
    
    beta_arma = lsq(F, beta_initial,xtol=5*1e-5,ftol=5*1e-5,method='lm')[0]
    beta_bip = lsq(F_bip, beta_initial,xtol=5*1e-5,ftol=5*1e-5,method='lm')[0]
    
    a_sc = rsp.arma_s_resid_sc(x, beta_arma, p, q) # innovations m-scale for ARMA model
    a_bip_sc, x_filt = rsp.bip_s_resid_sc(x, beta_bip, p, q) # innovations m-scale for BIP-ARMA model
    
    # final parameter estimate uses the model that provides smaller
    # m-scale
    
    beta_hat = beta_arma if a_sc < a_bip_sc else beta_bip
    
    # final m-scale
    a_m_sc = min(a_sc, a_bip_sc)
    
    phi_bip_s = -1*beta_hat[:p] if p>0 else []
    
    phi_bip_s_init = -1* beta_initial[:p] if p>0 else []
    
    theta_bip_s = -1*beta_hat[p:] if q>0 else []
    
    theta_bip_s_init = -1* beta_initial[p:] if q>0 else []
    
    results = {'ar_coeffs': phi_bip_s,
              'ma_coeffs': theta_bip_s,
              'inno_scale': a_m_sc,
              'cleaned_signal': x_filt,
              'ar_coeffs_init': phi_bip_s_init,
              'ma_coeffs_init': theta_bip_s_init}
    
    return results

x = np.array([-0.1242, -2.5415 ,0.2772 ,-0.1960 ,-0.1962 ,-0.3057, -1.1289 ,0.1942 ,-0.6071, -0.8284 ,0.5358 ,0.1095 ,-1.1221, 0.0460, -1.2386, 0.6382, 1.1452, -0.0159, 0.6610 ,-2.5455])

arma_est_bip_s(x,2,1)

not stationary


c:\users\computer\desktop\robustsp\robustsp\DependentData\Auxiliary\m_scale.py:19: RuntimeWarning: invalid value encountered in double_scalars
  if np.abs(sigma_k_plus1/sigma_k-1) > epsilon:


ValueError: Method 'lm' doesn't work when the number of residuals is less than the number of variables.

   -0.1242
   -2.5415
    0.2772
   -0.1960
   -0.1962
   -0.3057
   -1.1289
    0.1942
   -0.6071
   -0.8284
    0.5358
    0.1095
   -1.1221
    0.0460
   -1.2386
    0.6382
    1.1452
   -0.0159
    0.6610
   -2.5455

In [1]:
import numpy as np
import robustsp as rsp
#from robustsp.DependentData.Auxiliary.helper import poles as poles

def arma_s_resid_sc(x, beta_hat, p, q):
    
    phi_hat = beta_hat[:p] if 0<p else []
    theta_hat = beta_hat[p:] if 0<q else [] 
     
    N = len(x)
    r = max(p,q)
    a = np.zeros(N)
    x_sc = rsp.m_scale(x)
    
    poles = lambda x: np.sum(np.abs(np.roots(-1*np.array([-1,*x])))>1)

    if r==0:
        a_sc = np.array(x_sc)
        a = np.array(x)
    else:
        if poles(phi_hat) or poles(theta_hat):
            print('not stationary')
            a_sc = 10**10
        elif p>=1 and q>=1:
            # ARMA Models
            print('ARMA')
            for ii in range(r,N):
                # ARMA residuals
                xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
                aArr = a[ii-1::-1] if ii-q-1 < 0 else a[ii-1:ii-q-1:-1]
                
                a[ii] = x[ii]-np.sum(phi_hat*xArr)+\
                np.sum(theta_hat*aArr)

        elif p==0 and q>=1:
            # MA model
            for ii in range(r,N):
                aArr = a[ii-1::-1] if ii-q-1 < 0 else a[ii-1:ii-q-1:-1]
                a[ii]=x[ii]+np.sum(theta_hat*aArr)
        elif p>=1 and q==0:
            # AR model
            for ii in range(r,N):
                xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
                a[ii] = x[ii] - np.sum(phi_hat*xArr) # AR residuals
    
    a_sc = rsp.m_scale(a[p:])
    
    return a_sc

x = np.array([.4153,-.6244,-1.8424,-1.655])
b = np.array([1,2,3])
p=1
q=2
#arma_s_resid_sc(x,b,1,2) # 1e10 not stationary, stimmt
# arma_s_resid_sc(x,[0.5,-0.5,0.5],1,2) # 0.0429 stimmt
# arma_s_resid_sc(x,[0.5,-0.5,0.5],2,1) stimmt
# arma_s_resid_sc(x,[0.5],0,1) stimmt
rsp.arma_s_resid_sc(x,[0.5],1,0)

1.4478144118798

In [43]:
np.array((1,2,3))@np.array((4,5,6))

32

In [1]:
import numpy as np
import robustsp as rsp

def bip_s_resid_sc(x, beta_hat, p, q):
    
    phi_hat = np.array(beta_hat[:p]) # [] if p=0, MA case
    
    theta_hat = np.array(beta_hat[p:]) # [] if p=length(beta_hat), AR case
    
    x = np.array(x)
    N = len(x)
    r = max(p,q)
    
    a_bip = np.zeros(N)
    
    x_sc = rsp.m_scale(x)
    
    kap2 = 0.8724286
    
    poles = lambda x: np.sum(np.abs(np.roots(-1*np.array([-1,*x])))>1)
    
    if poles(phi_hat) or poles(theta_hat):
        sigma_hat = x_sc
    
    else:
        lamb = rsp.ma_infinity(phi_hat, -1*theta_hat, 100) # MA infinity approximation to compute scale used in eta function
        sigma_hat = np.sqrt(x_sc**2/(1+kap2*np.sum(lamb**2))) # scale used in eta
        
    if r == 0:
        a_sc_bip = x_sc
        a_bip = np.array(x)
    else:
        if poles(phi_hat) or poles(theta_hat):
            a_bip_sc = 10**10
        elif p>=1 and q>=1:
            # ARMA model
            for ii in range(r,N):
                # BIP-ARMA residuals
                xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
                aArr = a_bip[ii-1::-1] if ii-q-1 < 0 else a_bip[ii-1:ii-q-1:-1]
                apArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]

                a_bip[ii] = x[ii]\
                -phi_hat@(xArr-apArr+sigma_hat*rsp.eta(apArr/sigma_hat))+sigma_hat*(theta_hat[None,:])@(rsp.eta(aArr/sigma_hat)[:,None])
                
        elif p==0 and q>=1:
            # MA residuals
            for ii in range(r,N):
                # BIP-MA residuals
                aArr = a_bip[ii-1::-1] if ii-q-1 < 0 else a_bip[ii-1:ii-q-1:-1]
                
                a_bip[ii] = x[ii]+theta_hat[None,:]@(sigma_hat*rsp.eta(aArr/sigma_hat)[:,None])

        elif p>=1 and q==0:
            # AR Model
            for ii in range(r,N):
                xArr = x[ii-1::-1] if ii-p-1 < 0 else x[ii-1:ii-p-1:-1]
                apArr = a_bip[ii-1::-1] if ii-p-1 < 0 else a_bip[ii-1:ii-p-1:-1]

                a_bip[ii] = x[ii]-phi_hat@(xArr-apArr\
                +sigma_hat*rsp.eta(apArr/sigma_hat))
                print(a_bip)

        a_bip_sc = rsp.m_scale(a_bip[p:])
        x_filt = np.array(x)
        
        for ii in range(p,N):
            x_filt[ii] = x[ii] - a_bip[ii] + sigma_hat * rsp.eta(a_bip[ii]/sigma_hat)
    
    return a_bip_sc, x_filt

x = np.array([.4153,-.6244,-1.8424,-1.655])

# bip_s_resid_sc(x,[0.5],1,0)
# bip_s_resid_sc(x,[0.5],0,1)
rsp.bip_s_resid_sc(x,[0.5,-0.5,0.5],2,1) #stimmt

(2.0915284912614376, array([ 0.4153, -0.6244, -1.8424, -1.655 ]))

In [4]:
a =[1,2,3]
1@np.array([0])

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)